<a href="https://colab.research.google.com/github/HarshaliD/Helmet-Detection-using-DL/blob/main/helmetDetection_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import mrcnn

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as KL
import tensorflow.keras.models as KM
from numpy import zeros, asarray
from xml.etree import ElementTree
from mrcnn.utils import Dataset, extract_bboxes
from mrcnn.visualize import display_instances
from matplotlib import pyplot

In [ ]:
import cv2
image_path = "C:\\Users\\lenovo\\Desktop\\archive (16)\\images\\0014.png"

# Read the image using OpenCV
image = cv2.imread(image_path)

# Check if image was read successfully
if image is not None:
  # Get the image size (height, width)
  height, width, channels = image.shape

  # Print the image size
  print(f"Image size: (height, width) = ({height}, {width})")
else:
  print(f"Error: Could not read image at path: {image_path}")

Image size: (height, width) = (415, 416)


In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
class FruitsDataset(Dataset):
    # load the dataset definitions
    def load_dataset_annot(self, dataset_dir, is_train=True):
        # define classes
        self.add_class("dataset", 1, "helmet")

        # define data locations
        images_dir = os.path.join(dataset_dir, 'images')
        annotations_dir = os.path.join("C:\\Users\\lenovo\\Desktop\\archive (16)", 'annotations')

        # find all images
        for filename in os.listdir(annotations_dir):
            print(filename)
			# extract image id
            image_id = filename.split()[0][:4]
			#print('IMAGE ID: ',image_id)

			# skip all images after 115 if we are building the train set
            if is_train and int(image_id) >= 2000:
                continue
			# skip all images before 115 if we are building the test/val set
            if not is_train and int(image_id) < 2000:
                continue
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [1])


    def extract_boxes(self, filename, resize_factor=(1.0, 1.0)):
    # Load and parse the file
        tree = ElementTree.parse(filename)
        # Get the root of the document
        root = tree.getroot()

        # Extract each bounding box
        boxes = list()
        for box in root.findall('.//object'):
            name = box.find('name').text  # Add label name to the box list
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)

            # Resize coordinates based on the factors
            resized_xmin = int(xmin * resize_factor[0])
            resized_ymin = int(ymin * resize_factor[1])
            resized_xmax = int(xmax * resize_factor[0])
            resized_ymax = int(ymax * resize_factor[1])

            coors = [resized_xmin, resized_ymin, resized_xmax, resized_ymax, name]
            boxes.append(coors)

        # Extract original image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)

        return boxes, width, height


    def get_label(self, image_path):
        """
      Extracts labels and image dimensions from the corresponding XML annotation file.

      Args:
        image_path (str): Path to the image file.

      Returns:
        list: List of labels extracted from the XML file.
        int: Width of the original image.
        int: Height of the original image.
      """
        filename, _ = os.path.splitext(os.path.basename(image_path))
        annotation_file = os.path.join("C:\\Users\\lenovo\\Desktop\\archive (16)\\annotations", filename + '.xml')

        if not os.path.exists(annotation_file):
            raise FileNotFoundError(f"Annotation file not found: {annotation_file}")

        try:
            # Calculate resize factor dynamically based on target size and original image dimensions
            original_width, original_height = get_image_dimensions(image_path)
            resize_factor_w = target_size[0] / original_width
            resize_factor_h = target_size[1] / original_height

            # Use extract_boxes to get labels and image dimensions
            boxes, width, height = self.extract_boxes(annotation_file, resize_factor=(resize_factor_w, resize_factor_h))

            # Return labels, original width, and height
            labels = [box[-1] for box in boxes]  # Extract only labels from bounding boxes
            return labels, original_width, original_height

        except Exception as e:
            print(f"Error parsing annotation file: {annotation_file} - {e}")
            return [], 0, 0  # Handle parsing errors (optional)

    import random

    def load_dataset(self, dataset_dir, val_ratio=0.2, test_ratio=0.1):
        """
        Loads dataset images and annotations, splitting into train, validation, and test sets.

        Args:
            dataset_dir (str): Path to the dataset directory.
            val_ratio (float, optional): Ratio for splitting the dataset into validation (default: 0.2).
            test_ratio (float, optional): Ratio for splitting the dataset into test (default: 0.1).
        """

        # Define classes
        self.add_class("dataset", 1, "helmet")

        # Define data locations
        images_dir = os.path.join(dataset_dir, 'images')
        annotations_dir = os.path.join("C:\\Users\\lenovo\\Downloads\\archive (16)", 'annotations')

        all_image_ids = []
        train_image_ids = []
        val_image_ids = []
        test_image_ids = []
        train_annotated = []
        val_annotated = []
        test_annotated = []

        # Find all images
        for filename in os.listdir(images_dir):
            image_id = filename.split("_", 1)[0][:4]
            all_image_ids.append(image_id)

        # Split data into train/val/test based on ratios
        total_images = len(all_image_ids)
        num_test_images = int(total_images * test_ratio)
        num_val_images = int(total_images * val_ratio)
        num_train_images = total_images - (num_test_images + num_val_images)

        # Randomly select image IDs for test and validation sets
        test_image_ids = random.sample(all_image_ids, num_test_images)
        remaining_ids = [image_id for image_id in all_image_ids if image_id not in test_image_ids]
        val_image_ids = random.sample(remaining_ids, num_val_images)
        train_image_ids = [image_id for image_id in remaining_ids if image_id not in val_image_ids]

        # Process training set images
        for image_id in train_image_ids:
            img_path = os.path.join(images_dir, image_id + ".png")  # Adjust image extension if needed
            ann_path = os.path.join(annotations_dir, image_id + '.xml')  # Modify annotation path if needed

            # Add image to the dataset (assuming your function handles annotations internally)
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids=[1])

            # Get labels, width, and height using get_label (assuming it works for your data structure)
            labels, width, height = self.get_label(img_path)

            # Append image ID, labels, width, and height to train_annotated list
            train_annotated.append((image_id, labels, width, height))

        # Process validation and test sets with similar logic, storing in separate lists
        for image_id in val_image_ids:
            img_path = os.path.join(images_dir, image_id + ".png")
            ann_path = os.path.join(annotations_dir, image_id + '.xml')
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids=[1])
            labels, width, height = self.get_label(img_path)
            val_annotated.append((image_id, labels, width, height))

        for image_id in test_image_ids:
            img_path = os.path.join(images_dir, image_id + ".png")
            ann_path = os.path.join(annotations_dir, image_id + '.xml')
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids=[1])
            labels, width, height = self.get_label(img_path)
            test_annotated.append((image_id, labels, width, height))

        # Convert to numpy arrays
        train_annotated_np = np.array(train_annotated, dtype=object)
        val_annotated_np = np.array(val_annotated, dtype=object)
        test_annotated_np = np.array(test_annotated, dtype=object)

        return train_image_ids, val_image_ids, test_image_ids, train_annotated_np, val_annotated_np, test_annotated_np

    def load_image(self):
            val = "C:\\Users\\lenovo\\Desktop\\archive (16)\\images"
            """
            Loads and preprocesses an image from the given path.
            Args:
                img_path (str): Path to the image file.
            Returns:
                numpy.ndarray: Preprocessed image array.
            """
            image = load_img(val, target_size=(224, 224))  # Adjust target size as needed
            image = img_to_array(image)
            image = image.astype('float32') / 255.0
            return image


    def load_images_and_labels(self, image_ids, annotations, images_dir):
        images = []
        labels = []
        for annotation in annotations:
            image_id, label, width, height = annotation
            img_path = os.path.join(images_dir, image_id + ".png")
            image = self.load_image()
            images.append(image)
            labels.append(label)
        return np.array(images, dtype='float32'), np.array(labels, dtype='int32')  # Ensure consistent dtype



# Example usage
dataset_dir = "C:\\Users\\lenovo\\Desktop\\archive (16)"
train_set = FruitsDataset(dataset_dir)
train_image_ids, val_image_ids, test_image_ids, train_annotated, val_annotated, test_annotated = train_set.load_dataset(dataset_dir,val_ratio=0.2, test_ratio=0.1)
train_images, train_labels = train_set.load_images_and_labels(train_image_ids, train_annotated,dataset_dir)
val_images, val_labels = train_set.load_images_and_labels(val_image_ids, val_annotated, dataset_dir)
# Access training, validation, and test image IDs for further processing
print("Train:", len(train_image_ids))
print("Validation:", len(val_image_ids))
print("Test:", len(test_image_ids))
print("Test:", len(test_annotated))




Error parsing annotation file: C:\Users\lenovo\Desktop\archive (16)\annotations\0001.xml - name 'get_image_dimensions' is not defined
Error parsing annotation file: C:\Users\lenovo\Desktop\archive (16)\annotations\0002.xml - name 'get_image_dimensions' is not defined
Error parsing annotation file: C:\Users\lenovo\Desktop\archive (16)\annotations\0003.xml - name 'get_image_dimensions' is not defined
Error parsing annotation file: C:\Users\lenovo\Desktop\archive (16)\annotations\0004.xml - name 'get_image_dimensions' is not defined
Error parsing annotation file: C:\Users\lenovo\Desktop\archive (16)\annotations\0006.xml - name 'get_image_dimensions' is not defined
Error parsing annotation file: C:\Users\lenovo\Desktop\archive (16)\annotations\0007.xml - name 'get_image_dimensions' is not defined
Error parsing annotation file: C:\Users\lenovo\Desktop\archive (16)\annotations\0010.xml - name 'get_image_dimensions' is not defined
Error parsing annotation file: C:\Users\lenovo\Desktop\archive

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\lenovo\\Desktop\\archive (16)\\images'

In [ ]:

image_path = "C:\\Users\\lenovo\\Downloads\\archive (14)\\images\\3946.png"

# Read the image using OpenCV
image = cv2.imread(image_path)

# Check if image was read successfully
if image is not None:
  # Get the image size (height, width)
  height, width, channels = image.shape

  # Print the image size
  print(f"Image size: (height, width) = ({height}, {width})")
else:
  print(f"Error: Could not read image at path: {image_path}")

Image size: (height, width) = (224, 224)


In [ ]:
from tensorflow.keras.applications import ResNet50

# Load the pre-trained ResNet50 model with ImageNet weights
model = ResNet50(weights='imagenet', include_top=False,input_shape=(224, 224, 3))


In [ ]:
print(model.output.shape)

(None, 7, 7, 2048)


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten,Dense

In [ ]:
x = model.output
x = Flatten()(x)  # Flatten the output (optional)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Assuming binary classification


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Assuming binary classification


In [ ]:
history = model.fit(train_images, train_labels,
                            epochs=10,  # Adjust number of epochs as needed
                            batch_size=32,  # Adjust batch size as needed
                            validation_data=(val_images, val_labels))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).